In [ ]:
import datetime, time
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

from neo4j import GraphDatabase
import nxneo4j as nx
import csv
import io
import os
import warnings;
import urllib.request
import urllib, urllib.parse
import json
from datetime import datetime, timedelta
import os
import urllib.parse
import csv
import unicodedata
import io
import time
import numpy as np 
# output results of multiple statements in one cell
from IPython.core.interactiveshell import InteractiveShell

# load libraries
from multiprocessing import freeze_support
from multiprocessing import Pool
from functools import partial
import threading
import pandas as pd
import numpy as np
from random import randint
from time import sleep
import datetime, xlrd
import spotipy
import spotipy.util as util
import pandas as pd # for later
import spotipy.oauth2 as oauth2
import time
import csv
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON
import datetime, xlrd
import math
from joblib import Parallel, delayed
from scipy.sparse import coo_matrix
import array
import numpy as np

warnings.filterwarnings('ignore')

def unique(list1): 
    x = np.array(list1) 
    return np.unique(x) 

In [ ]:
file_path_tracks_features1 ="..//a_fm_data_prep_modif_user_item_tfidf_final.csv"

df_tracks_features1 = pd.read_csv(file_path_tracks_features1 , encoding= 'ISO-8859-1')

users = df_tracks_features1["user_id"].unique().tolist()
user_count = len(users)

urs = [users[i] for i in range(round(len(users)))] #0.8 ,0.5
df = df_tracks_features1[df_tracks_features1['user_id'].isin(urs)]

tracks = []
for i in tqdm(urs):
    temp = df[df["user_id"] == i]["item_id"].tolist()
    tracks.append(temp)  

weight_all = []
for i in tqdm(urs):
    temp = df[df["user_id"] == i]["weight_all"].tolist()
    weight_all.append(temp)  

weight_content = []
for i in tqdm(urs):
    temp = df[df["user_id"] == i]["weight_content"].tolist()
    weight_content.append(temp)  

weight_context = []
for i in tqdm(urs):
    temp = df[df["user_id"] == i]["weight_context"].tolist()
    weight_context.append(temp)  

interaction_description = []
for i in tqdm(urs):
    temp = df[df["user_id"] == i]["interaction_description"].tolist()
    interaction_description.append(temp)  


In [ ]:
interactions = pd.DataFrame({'user_id': urs,
                             'item_id': tracks})

In [ ]:
# To build the Factorization Machines model, we use LightFm libraries. 
# For more information about the libraries, you can visit the documentation page below:
# https://making.lyst.com/lightfm/docs/home.html

from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score
from lightfm.data import Dataset
from scipy.sparse import *
from scipy import *

urs  = np.array(interactions['user_id'])
tracks  = np.array(interactions['item_id'])

train_FM_ides = [1 for i in range(0,len(urs))]

n = max(max(urs), max(tracks)) + 1
interactions = coo_matrix((train_FM_ides, (urs, tracks)),shape=(n, n)) 


In [ ]:
from unittest import mock

rs = np.random.RandomState(3)
interactions_train_FM, interactions_val_FM = random_train_test_split(interactions, test_percentage=0.2, random_state=rs)

In [ ]:
user_train_ids_FM, track_train_ids_FM = interactions_train_FM.nonzero()
user_val_ids_FM, track_val_ids_FM = interactions_val_FM.nonzero()

In [ ]:
urs_train = user_train_ids_FM
tracks_train = track_train_ids_FM

In [ ]:
n = max(max(urs_train), max(tracks_train)) + 1
interactions_train = coo_matrix((train_FM_ides, (urs_train, tracks_train)),shape=(n, n)) 
interactions_all_train_weight = coo_matrix((weight_all_train, (urs_train, tracks_train)),shape=(n, n)) 
interactions_content_train_weight = coo_matrix((weight_content_train, (urs_train, tracks_train)),shape=(n, n)) 
interactions_context_train_weight = coo_matrix((weight_context_train, (urs_train, tracks_train)),shape=(n, n)) 



In [ ]:
no_c=30
ep=20
loss_= 'bpr' #warp
l_r=0.005
k_=10

In [ ]:
model = LightFM(no_components=no_c, loss=loss_, learning_rate=l_r)
model.fit(interactions_train_FM, 
          epochs=ep)

predictions1 = model.predict(user_val_ids_FM, track_val_ids_FM) 

train_precision_1 = precision_at_k(model, interactions_train_FM, k=k_).mean()
test_precision_1 = precision_at_k(model, interactions_val_FM).mean()

train_recall_1 = recall_at_k(model, interactions_train_FM,  k=k_).mean()
test_recall_1 = recall_at_k(model, interactions_val_FM).mean()

print("-------------------------------result----------------------------------")
print("General (without weights):")
print("train_precision_1 : ", train_precision_1)
print("test_precision_1 : ", test_precision_1)
print("train_recall_1 : ", train_recall_1)
print("test_recall_1 : ", test_recall_1)

In [ ]:
model.fit(interactions_train_FM, 
          sample_weight= interactions_all_train_weight, 
          epochs=ep)

predictions2 = model.predict(user_val_ids_FM, track_val_ids_FM) 

train_precision_2 = precision_at_k(model, interactions_train_FM, k=k_).mean()
test_precision_2 = precision_at_k(model, interactions_val_FM).mean()

train_recall_2 = recall_at_k(model, interactions_train_FM,  k=k_).mean()
test_recall_2 = recall_at_k(model, interactions_val_FM).mean()

print("With all weights:")
print("train_precision_2 : ", train_precision_2)
print("test_precision_2 : ", test_precision_2)
print("train_recall_2 : ", train_recall_2)
print("test_recall_2 : ", test_recall_2)

In [ ]:
model.fit(interactions_train_FM, 
          sample_weight= interactions_content_train_weight,  
          epochs=ep)

predictions3 = model.predict(user_val_ids_FM, track_val_ids_FM) 

train_precision_3 = precision_at_k(model, interactions_train_FM, k=k_).mean()
test_precision_3 = precision_at_k(model, interactions_val_FM).mean()

train_recall_3 = recall_at_k(model, interactions_train_FM,  k=k_).mean()
test_recall_3 = recall_at_k(model, interactions_val_FM).mean()

print("With content weights:")
print("train_precision_3 : ", train_precision_3)
print("test_precision_3 : ", test_precision_3)
print("train_recall_3 : ", train_recall_3)
print("test_recall_3 : ", test_recall_3)

In [ ]:
model.fit(interactions_train_FM, 
          sample_weight= interactions_context_train_weight,  
          epochs=ep)

predictions4 = model.predict(user_val_ids_FM, track_val_ids_FM) 

train_precision_4 = precision_at_k(model, interactions_train_FM, k=k_).mean()
test_precision_4 = precision_at_k(model, interactions_val_FM).mean()


train_recall_4 = recall_at_k(model, interactions_train_FM,  k=k_).mean()
test_recall_4 = recall_at_k(model, interactions_val_FM).mean()

print("With context weights:")
print("train_precision_4 : ", train_precision_4)
print("test_precision_4 : ", test_precision_4)
print("train_recall_4 : ", train_recall_4)
print("test_recall_4 : ", test_recall_4)